In [ ]:
import modules.widget_framework as framework
from IPython.html import widgets
from IPython.display import display, clear_output
reload(framework)

In [ ]:
frame = framework.framework()
frame.set_default_display_style(box_style="info")
frame.set_default_io_style(padding="0.5em", margin="0.5em")

In [ ]:
states_nugrid = ["nugrid", "nugrid_w_data", "iso_abund", "abu_chart", "movie"]
states_mesa = ["mesa", "mesa_w_data", "hrd", "plot"]
states_plotting = states_nugrid[2:]+states_mesa[2:]

frame.add_state(states_nugrid)
frame.add_state(states_mesa)

frame.add_display_object("window")
frame.add_io_object("Title")
frame.add_display_object("widget")

###Data page###
frame.add_display_object("page_data")
frame.add_io_object("mass")
frame.add_io_object("Z")
frame.add_io_object("select_nugrid_mesa")

frame.add_display_object("contain_module_load")
frame.add_io_object("select_module")
frame.add_io_object("load_data")

frame.add_io_object("select_plot")

frame.set_state_children("window", ["Title", "widget"])
frame.set_state_children("widget", ["page_data"])
frame.set_state_children("page_data", ["mass", "Z", "select_nugrid_mesa",
                                       "contain_module_load", "select_plot"])
frame.set_state_children("contain_module_load", ["select_module", "load_data"])

###Plotting page###
frame.add_display_object("page_plotting")

frame.add_io_object("warning_msg")

frame.add_io_object("plot_name")
frame.add_io_object("cycle")
frame.add_io_object("cycle_range")

frame.add_display_object("xax")
frame.add_io_object("xaxis")
frame.add_io_object("logx")
frame.add_display_object("yax")
frame.add_io_object("yaxis")
frame.add_io_object("logy")

frame.add_display_object("mass_settings")
frame.add_io_object("set_amass")
frame.add_io_object("amass_range")
frame.add_io_object("set_mass")
frame.add_io_object("mass_range")
frame.add_io_object("lbound")

frame.add_display_object("lim_settings")
frame.add_io_object("set_lims")
frame.add_io_object("ylim")
frame.add_io_object("xlim")

frame.add_io_object("stable")

frame.add_display_object("abu_settings")
frame.add_io_object("ilabel")
frame.add_io_object("imlabel")
frame.add_io_object("imagic")

frame.add_io_object("generate_plot")

frame.set_state_children("widget", ["page_plotting"])
frame.set_state_children("page_plotting", ["warning_msg", "plot_name","cycle", "cycle_range",
                                           "xax", "yax", "stable", "mass_settings",
                                           "lim_settings", "abu_settings", "stable", "generate_plot"])
frame.set_state_children("xax", ["xaxis", "logx"])
frame.set_state_children("yax", ["yaxis", "logy"])
frame.set_state_children("mass_settings", ["set_amass", "amass_range", "set_mass", "mass_range",
                                           "lbound"])
frame.set_state_children("lim_settings", ["set_lims", "xlim", "ylim"])
frame.set_state_children("abu_settings", ["ilabel", "imlabel", "imagic"])

In [ ]:
###DEFAULT###

frame.set_state_data("class_instance", None)

frame.set_state_attribute('window', visible=True)
frame.set_state_attribute('Title', visible=True, value = "<h1>NuGrid / Mesa Explorer</h1>")
frame.set_state_attribute('widget', visible=True)

frame.set_state_attribute("page_data", visible=True)
frame.set_state_attribute('mass', visible=True, description="mass", min=0,
                          max=25, step=0.25)
frame.set_state_attribute('Z', visible=True, description="Z",
                          options=["1E-4", "1E-3", "6E-3", "1E-2", "2E-2"])
frame.set_state_attribute("select_nugrid_mesa", visible=True,
                          description="Select NuGrid or Mesa",
                          options=["", "NuGrid", "Mesa"])
frame.set_state_attribute("contain_module_load", visible=True)
frame.set_state_attribute("select_module", visible=True,
                          description="Select data type", disabled=True)
frame.set_state_attribute("load_data", visible=True,
                          description="Load Data", disabled=True)
frame.set_state_attribute("select_plot", visible=True,
                          description="Select plot type", disabled=True)

###NUGRID###
frame.set_state_attribute("select_module", states_nugrid, options={"H5 out":"se"}, disabled=False)
frame.set_state_attribute("load_data", states_nugrid, disabled=False)
frame.set_state_attribute("select_plot", states_nugrid[1:], options={"Iso abundance":"iso_abund", "abundance chart":"abu_chart", "movie":"movie"}, disabled=False)

###MESA###
frame.set_state_attribute("select_module", states_mesa, options={"History":"star_log", "Profile":"mesa_profile"}, disabled=False)
frame.set_state_attribute("load_data", states_mesa, disabled=False)
frame.set_state_attribute("select_plot", states_mesa[1:], options={"HR-Diagram":"hrd", "Plot":"plot"}, disabled=False)

###CALLBACKS###
def sel_nugrid_mesa(widget, value):
    if value=="NuGrid":
        frame.set_state("nugrid")
    elif value=="Mesa":
        frame.set_state("mesa")
    elif value=="":
        frame.set_state("default")

def load(widget):
    clear_output()
    data = None
    mass = frame.get_attribute("mass", "value")
    Z = float(frame.get_attribute("Z", "value"))
    module = frame.get_attribute("select_module", "value")
    if module == "se":
        data = mp.se(mass=mass, Z=Z)
        frame.set_state("nugrid_w_data")
    elif module == "star_log":
        data = ms.history_data(mass=mass, Z=Z)
        frame.set_state("mesa_w_data")
        frame.set_attributes("xaxis", options=data.cols.keys())
        frame.set_attributes("yaxis", options=data.cols.keys())
    elif module == "mesa_profile":
        data = ms.mesa_profile(mass=mass, Z=Z)
        frame.set_state("mesa_w_data")
        frame.set_attributes("xaxis", options=data.cols.keys())
        frame.set_attributes("yaxis", options=data.cols.keys())
    else:
        frame.set_state("default")
    frame.set_state_data("class_instance", data)
        
def sel_plot(widget, value):
    frame.set_state(value)
    
def change_module(widget, value):
    if value == "star_log":
        frame.set_state_attribute("select_plot", states_mesa[1:], options={"HR-Diagram":"hrd", "Plot":"plot"})
    elif value == "mesa_profile":
        frame.set_state_attribute("select_plot", states_mesa[1:], options={"Plot":"plot"})

frame.set_state_callbacks("select_nugrid_mesa", sel_nugrid_mesa)
frame.set_state_callbacks("select_module", change_module)
frame.set_state_callbacks("load_data", load, attribute=None, type="on_click")
frame.set_state_callbacks("select_plot", sel_plot)

frame.set_object("window", widgets.Box())
frame.set_object("Title", widgets.HTML())
frame.set_object("widget", widgets.Tab())

frame.set_object("page_data", widgets.VBox())
frame.set_object("mass", widgets.FloatSlider())
frame.set_object("Z", widgets.ToggleButtons())

frame.set_object("select_nugrid_mesa", widgets.Dropdown())
frame.set_object("contain_module_load", widgets.HBox())
frame.set_object("select_module", widgets.Dropdown())
frame.set_object("load_data", widgets.Button())
frame.set_object("select_plot", widgets.Dropdown())

In [ ]:

frame.set_state_attribute('page_plotting', visible=True)

frame.set_state_attribute('warning_msg', visible=True, value="<h3>Error: No data loaded!</h3>")
frame.set_state_attribute("warning_msg", states_plotting, visible=False)

frame.set_state_attribute('plot_name', "iso_abund", visible=True, value="<h2>Isotope abundance</h2>")
frame.set_state_attribute('plot_name', "abu_chart", visible=True, value="<h2>Abundance chart</h2>")
frame.set_state_attribute('plot_name', "movie", visible=True, value="<h2>Movie</h2>")
frame.set_state_attribute('plot_name', "hrd", visible=True, value="<h2>HR-Diagram</h2>")
frame.set_state_attribute('plot_name', "plot", visible=True, value="<h2>Plot</h2>")

frame.set_state_attribute('cycle', ["iso_abund", "abu_chart"], visible=True, description="cycle: ")
frame.set_state_attribute('cycle_range', ["movie"], visible=True)

frame.set_state_attribute('xax', "plot", visible=True)
frame.set_state_attribute('xaxis', visible=True, description="select X-axis: ")
frame.set_state_attribute('logx', visible=True, description="log X-axis: ")
frame.set_state_attribute('yax', "plot", visible=True)
frame.set_state_attribute('yaxis', visible=True, description="select Y-axis: ")
frame.set_state_attribute('logy', visible=True, description="log Y-axis: ")

frame.set_state_attribute("mass_settings", ["iso_abund", "abu_chart"], visible=True)
frame.set_state_attribute("set_amass", "iso_abund", visible=True, description="Set amass range: ")
frame.set_state_attribute("amass_range", "iso_abund", description="amass range: ")
frame.set_state_attribute("set_mass", ["iso_abund", "abu_chart"], visible=True, description="Set mass range: ")
frame.set_state_attribute("mass_range", ["iso_abund", "abu_chart"], description="mass range: ")
frame.set_state_attribute("lbound", "abu_chart", visible=True, description="lbound", min=-12, max=0)

frame.set_state_links("amass_link", [("set_amass", "value"), ("amass_range", "visible")], "iso_abund", True)
frame.set_state_links("mass_link", [("set_mass", "value"), ("mass_range", "visible")], ["iso_abund", "abu_chart"], True)

frame.set_state_attribute("lim_settings" , ["iso_abund", "abu_chart"], visible=True)
frame.set_state_attribute("set_lims", ["iso_abund", "abu_chart"], visible=True, description="Set axis limits: ")
frame.set_state_attribute("xlim", "abu_chart", description="x-axis limits: ")
frame.set_state_attribute("ylim", ["iso_abund", "abu_chart"], description="y-axis limits: ")

frame.set_state_links("xlims_link", [("set_lims", "value"), ("xlim", "visible")], "abu_chart", True)
frame.set_state_links("ylims_link", [("set_lims", "value"), ("ylim", "visible")], ["iso_abund", "abu_chart"], True) 

frame.set_state_attribute("abu_settings", "abu_chart", visible=True)
frame.set_state_attribute("ilabel", "abu_chart", visible=True, description="ilabel")
frame.set_state_attribute("imlabel", "abu_chart", visible=True, description="imlabel")
frame.set_state_attribute("imagic", "abu_chart", visible=True, description="imagic")

frame.set_state_attribute("stable", "iso_abund", visible=True, description="stable: ")

frame.set_state_attribute('generate_plot', states_plotting, visible=True, description="Generate Plot")


def make_plot(widget):
    state = frame.get_state()
    
    data = frame.get_state_data("class_instance")
    cycle = frame.get_attribute("cycle", "value")
    cycle_range = frame.get_attribute("cycle_range", "value")
    xax = frame.get_attribute("xaxis", "value")
    logx = frame.get_attribute("logx", "value")
    yax = frame.get_attribute("yaxis", "value")
    logy = frame.get_attribute("logy", "value")
    if frame.get_attribute("set_amass", "value"):
        amass = frame.get_attribute("amass_range", "value")
    else:
        amass = None
        
    if frame.get_attribute("set_mass", "value"):
        mass = frame.get_attribute("mass_range", "value")
    else:
        mass = None
        
    lbound = frame.get_attribute("lbound", "value")
        
    if frame.get_attribute("set_lims", "value"):
        xlim = frame.get_attribute("xlim", "value")
        ylim = frame.get_attribute("ylim", "value")
    else:
        xlim = [0, 0]
        ylim = [0, 0]
        
    stable = frame.get_attribute("stable", "value")
    ilabel = frame.get_attribute("ilabel", "value")
    imlabel = frame.get_attribute("imlabel", "value")
    imagic = frame.get_attribute("imagic", "value")
        
    if state=="iso_abund":
        data.iso_abund(cycle, stable, amass, mass, ylim)
    elif state=="abu_chart":
        plotaxis = [xlim[0], xlim[1], ylim[0], ylim[1]]
        data.abu_chart(cycle, mass, ilabel, imlabel, imagic=imagic, lbound=lbound, plotaxis=plotaxis)
    elif state=="plot":
        print xax, yax
        data.plot(xax, yax, logx=logx, logy=logy)
    elif state=="hrd":
        data.hrd_new()

frame.set_state_callbacks("generate_plot", make_plot, attribute=None, type="on_click")

frame.set_object("page_plotting", widgets.VBox())
frame.set_object("warning_msg", widgets.HTML())
frame.set_object("plot_name", widgets.HTML())
frame.set_object("cycle", widgets.IntSlider())
frame.set_object("cycle_range", widgets.IntRangeSlider())
frame.set_object("xax", widgets.HBox())
frame.set_object("xaxis", widgets.Select())
frame.set_object("logx", widgets.Checkbox())
frame.set_object("yax", widgets.HBox())
frame.set_object("yaxis", widgets.Select())
frame.set_object("logy", widgets.Checkbox())

frame.set_object("mass_settings", widgets.VBox())
frame.set_object("set_amass", widgets.Checkbox())
frame.set_object("amass_range", widgets.IntRangeSlider())
frame.set_object("set_mass", widgets.Checkbox())
frame.set_object("mass_range", widgets.IntRangeSlider())
frame.set_object("lbound", widgets.FloatRangeSlider())

frame.set_object("lim_settings", widgets.VBox())
frame.set_object("set_lims", widgets.Checkbox())
frame.set_object("ylim", widgets.FloatRangeSlider())
frame.set_object("xlim", widgets.FloatRangeSlider())

frame.set_object("stable", widgets.Checkbox())

frame.set_object("abu_settings", widgets.HBox())
frame.set_object("ilabel", widgets.Checkbox())
frame.set_object("imlabel", widgets.Checkbox())
frame.set_object("imagic", widgets.Checkbox())

frame.set_object("generate_plot", widgets.Button())

In [ ]:
frame.display_object("window")